In [1]:
# First, make sure you have the necessary library installed
#!pip install PyMySQL --upgrade

In [2]:
import pymysql
import os
from dotenv import load_dotenv

#method to get values of environment varaibles from .env file 
#load_dotenv('.env')


# Environment variables for sensitive data
DB_HOST = "database-1.cnq4kwysqjdb.us-east-1.rds.amazonaws.com"
DB_PORT = 3306
DB_NAME = "mission_green"
DB_USER = ""
DB_PASSWORD = ""


# Connect to MySQL database
conn = pymysql.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    db=DB_NAME
)

# Create a cursor to execute SQL statements
cursor = conn.cursor()

# Create schema if it doesn't exist
schema_name = "mission_green_dw"
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {schema_name}")

# Table date_dim --------
table_name_date = 'date_dim'

# SQL command to create the date_dim table
create_table_date_dim = f"""
CREATE TABLE IF NOT EXISTS {schema_name}.{table_name_date} (
    date_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    date DATE,
    month INT,
    year INT,
    day INT
);
"""

# Execute the SQL command
cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name_date}")
cursor.execute(create_table_date_dim)

#Table gas_dim -------
table_name_gas = 'gas_dim'

# SQL command to create the gas_dim table
create_table_gas_dim = f"""
CREATE TABLE IF NOT EXISTS {schema_name}.{table_name_gas} (
    gas_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    aqs_parameter_code INT,
    aqs_parameter_desc VARCHAR(50)
);
    """

# Execute the SQL command
cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name_gas}")
cursor.execute(create_table_gas_dim)


#Table coliform_dim -------
table_name_coliform = 'coliform_dim'

# SQL command to create the coliform_dim table
create_table_coliform_dim = f"""
CREATE TABLE IF NOT EXISTS {schema_name}.{table_name_coliform} (
    coliform_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    label VARCHAR(15),
    code CHAR(2)
);
    """

# Execute the SQL command
cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name_coliform}")
cursor.execute(create_table_coliform_dim)


# Table ecoli_dim -------
table_name_ecoli = 'ecoli_dim'

# SQL command to create the ecoli_dim table
create_table_ecoli_dim = f"""
CREATE TABLE IF NOT EXISTS {schema_name}.{table_name_ecoli} (
    ecoli_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    label VARCHAR(15),
    code CHAR(2)
);
    """

# Execute the SQL command
cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name_ecoli}")
cursor.execute(create_table_ecoli_dim)


# Table meetstandards_dim -------
table_name_ms = 'meetstandards_dim'

# SQL command to create the meetstandards_dim table
create_table_ms_dim = f"""
CREATE TABLE IF NOT EXISTS {schema_name}.{table_name_ms} (
    ms_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    label VARCHAR(15),
    code CHAR(3)
);
    """

# Execute the SQL command
cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name_ms}")
cursor.execute(create_table_ms_dim)


# Table location_dim -------
table_name_location = 'location_dim'

# SQL command to create the location_dim table
create_table_location_dim = f"""
CREATE TABLE IF NOT EXISTS {schema_name}.{table_name_location} (
    location_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(100),
    state_code INT,
    county VARCHAR(100),
    county_code INT,
    zipcode INT
);
    """

# Execute the SQL command
cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name_location}")
cursor.execute(create_table_location_dim)



# Table air_water_fact -------
table_name_air = 'air_water_fact'

# SQL command to create the air_water_fact table
create_table_fact = f"""
CREATE TABLE {schema_name}.{table_name_air} (
  date_id int DEFAULT NULL,
  location_id int DEFAULT NULL,
  CO_Gas  float DEFAULT NULL,
  NO2_Gas  float DEFAULT NULL,
  Ozone_Gas  float DEFAULT NULL,
  PM10_Gas  float DEFAULT NULL,
  SO2_Gas  float DEFAULT NULL,
  PM2_Gas  float DEFAULT NULL,
  Max_AQI_Conc  float DEFAULT NULL,
  Gas_id int DEFAULT NULL,
  ecoli_id int DEFAULT NULL,
  coliform_id int DEFAULT NULL,
  ms_id int DEFAULT NULL,
  CONSTRAINT air_water_fact_ibfk_1 FOREIGN KEY (date_id) REFERENCES mission_green_dw.date_dim (date_id),
  CONSTRAINT air_water_fact_ibfk_2 FOREIGN KEY (location_id) REFERENCES mission_green_dw.location_dim (location_id),
  CONSTRAINT air_water_fact_ibfk_3 FOREIGN KEY (gas_id) REFERENCES mission_green_dw.gas_dim (gas_id),
  CONSTRAINT air_water_fact_ibfk_4 FOREIGN KEY (ecoli_id) REFERENCES mission_green_dw.ecoli_dim (ecoli_id),
  CONSTRAINT air_water_fact_ibfk_5 FOREIGN KEY (coliform_id) REFERENCES mission_green_dw.coliform_dim (coliform_id),
  CONSTRAINT air_water_fact_ibfk_6 FOREIGN KEY (ms_id) REFERENCES mission_green_dw.meetstandards_dim (ms_id)
);
    """

# Execute the SQL command
cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name_air}")
cursor.execute(create_table_fact)

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()
